In [15]:
import os
import numpy as np
from PIL import Image
import cv2
from matplotlib import pyplot as plt
import math as math
#!pip install imgaug
import imgaug.augmenters as iaa

# Charger les images

In [16]:
def load_images(path, gray_scale = False):
    """
    On charge les images et on le mets toutes avec les mêmes dimensions
    """
    images = []
    names = os.listdir(path)
    for f in os.listdir(path):
        img = None
        if gray_scale : 
            img = cv2.imread(os.path.join(path, f), 0)
        else :
            img = cv2.imread(os.path.join(path, f))
        if img is not None:
            img = cv2.resize(img, (1024,1024))
            images.append(img)
    s = sorted(zip(names,images))
    tuples = zip(*s)
    names , images = [list(tuple) for tuple in tuples]
    return images, names

def load_labels(path):
    """
    On charge les labels 
    """
    labels = []
    l_names = os.listdir(path)
    for f in os.listdir(path):
        with open(path+f,'r') as fp:
            ligne = fp.readline()
            liste = ligne.split(' ')
            labels.append(liste)
    s= sorted(zip(l_names,labels))
    tuples = zip(*s)
    l_names , labels = [list(tuple) for tuple in tuples]
    return labels, l_names

path_images = '../labelling/Images/Biliary_ducts/'
images,i_names = load_images(path_images, True)

path_labels = './labelling/Labels/output_Biliary_ducts/'
labels,l_names = load_labels(path_labels)

# afficher rectangle

In [18]:
def rectangle(img,label):
    # shape image
    n , m  = img.shape
    # coordonnées
    x_center = int(float(label[1]) * m)
    y_center = int(float(label[2]) * n)
    width = int(float(label[3]) * m)
    height = int(float(label[4]) * n)
    x1 = int(x_center - 0.5 * width)
    y1 = int(y_center - 0.5 * height)
    x2 = int(x_center + 0.5 * width)
    y2 = int(y_center  + 0.5 * height)
    # draw rectangle on image
    img_modif = img.copy()
    cv2.rectangle(img_modif, (x1, y1), (x2, y2), (0, 0, 255),thickness = 2)
    return img_modif

# Copier images et label vers le dossier souhaité (obligatoire)

In [42]:
def copier_image_original(labels,images,i_names,l_names,i_path,l_path):
    for i in range(len(images)):
        # Open the image and convert it to grayscale
        data = images[i]
        data = data.astype(np.uint8)
        # Save the stretched image to a new file
        imgpil = Image.fromarray(data) # Transformation du tableau en image PIL
        imgpil.save(i_path+l_names[i][:-4]+'.bmp')


    new_labels = labels
    for i in range(len(labels)):
        with open(l_path+l_names[i][:-4]+'.txt','w') as fp:
            ligne = f'{new_labels[i][0]} {new_labels[i][1]} {new_labels[i][2]} {new_labels[i][3]} {new_labels[i][4]}'
            fp.write(ligne)
            
    return True

In [43]:
i_path = '../piford/'
l_path = '../piford/'
copier_image_original(labels,images,i_names,l_names,i_path,l_path)

True

# Créer image et label rotation 180

In [19]:
def relabel_180(labels,images,i_names,l_names,i_path,l_path):
    n,m = images[0].shape
    new_labels = labels
    for i in range(len(labels)):
        new_labels[i][1] = (m - int(float(labels[i][1]) * m)) / m
        new_labels[i][2] = (n - int(float(labels[i][2]) * n)) / n
    
        with open(l_path+l_names[i][:-4]+'_r180.txt','w') as fp:
            ligne = f'{new_labels[i][0]} {new_labels[i][1]} {new_labels[i][2]} {new_labels[i][3]} {new_labels[i][4]}'
            fp.write(ligne)
    
    for i in range(len(images)):
        img_r180 = cv2.rotate(images[i],cv2.ROTATE_180)
        imgpil = Image.fromarray(img_r180) # Transformation du tableau en image PIL
        imgpil.save(i_path+l_names[i][:-4]+'_r180.bmp')
    return True



In [20]:
i_path = '../piford/'
l_path = '../piford/'
relabel_180(labels,images,i_names,l_names,i_path,l_path)

True

# créer image et label rotation 90 

In [21]:
def relabel_90(labels,images,i_names,l_names,i_path,l_path):
    n,m = images[0].shape
    new_labels = labels
    for i in range(len(labels)):
        x = labels[i][1]
        y = labels[i][2]
        w = labels[i][3]
        h = labels[i][4][:-1]
        x_prim = int(float(x) * 1024)-512
        y_prim = int(float(y) * 1024)-512
        xy = np.array([x_prim,y_prim]).reshape(-1,1)
        rot = np.array([[math.cos(3*math.pi/2), math.sin(3*math.pi/2)],[-math.sin(3*math.pi/2), math.cos(3*math.pi/2)]])
        mat = rot@xy
        x = (int(mat[0][0])+512)/1024
        y = (int(mat[1][0])+512)/1024
        new_labels[i][1] = x
        new_labels[i][2] = y
        new_labels[i][3] = h
        new_labels[i][4] = w   
        with open(l_path+l_names[i][:-4]+'_r90.txt','w') as fp:
            ligne = f'{new_labels[i][0]} {new_labels[i][1]} {new_labels[i][2]} {new_labels[i][3]} {new_labels[i][4]}\n'
            fp.write(ligne)
    
    for i in range(len(images)):
        img_r90 = cv2.rotate(images[i],cv2.ROTATE_90_CLOCKWISE)
        imgpil = Image.fromarray(img_r90) # Transformation du tableau en image PIL
        imgpil.save(i_path+l_names[i][:-4]+'_r90.bmp')
    return True

In [22]:
i_path = '../piford/'
l_path = '../piford/'
relabel_90(labels,images,i_names,l_names,i_path,l_path)

True

# creer image et label rotation 270

In [23]:
def relabel_270(labels,images,i_names,l_names,i_path,l_path):
    n,m = images[0].shape
    new_labels = labels
    for i in range(len(labels)):
        x = labels[i][1]
        y = labels[i][2]
        w = labels[i][3]
        h = labels[i][4][:-1]
        x_prim = int(float(x) * 1024)-512
        y_prim = int(float(y) * 1024)-512
        xy = np.array([x_prim,y_prim]).reshape(-1,1)
        rot = np.array([[math.cos(math.pi/2), math.sin(math.pi/2)],[-math.sin(math.pi/2), math.cos(math.pi/2)]])
        mat = rot@xy
        x = (int(mat[0][0])+512)/1024
        y = (int(mat[1][0])+512)/1024
        new_labels[i][1] = x
        new_labels[i][2] = y
        new_labels[i][3] = h
        new_labels[i][4] = w   
        with open(l_path+l_names[i][:-4]+'_r270.txt','w') as fp:
            ligne = f'{new_labels[i][0]} {new_labels[i][1]} {new_labels[i][2]} {new_labels[i][3]} {new_labels[i][4]}\n'
            fp.write(ligne)
    
    for i in range(len(images)):
        img_r270 = cv2.rotate(images[i],cv2.ROTATE_90_COUNTERCLOCKWISE)
        imgpil = Image.fromarray(img_r270) # Transformation du tableau en image PIL
        imgpil.save(i_path+l_names[i][:-4]+'_r270.bmp')
    return True

In [24]:
i_path = '../piford/'
l_path = '../piford/'
relabel_270(labels,images,i_names,l_names,i_path,l_path)

True

# créer image + label flip

In [25]:
def relabel_flip(labels,images,i_names,l_names,i_path,l_path):
    n,m = images[0].shape
    new_labels = labels
    for i in range(len(labels)):
        new_labels[i][1] = (m - int(float(labels[i][1]) * m)) / m
        with open(l_path+l_names[i][:-4]+'_flip.txt','w') as fp:
            ligne = f'{new_labels[i][0]} {new_labels[i][1]} {new_labels[i][2]} {new_labels[i][3]} {new_labels[i][4]}'
            fp.write(ligne)
    
    for i in range(len(images)):
        aug = iaa.Fliplr(1)
        img_flip = aug.augment_image(images[i])
        im = Image.fromarray(img_flip) # Transformation du tableau en image PIL
        im.save(i_path+l_names[i][:-4]+'_flip.bmp')
    return True

In [26]:
i_path = '../piford/'
l_path = '../piford/'
relabel_flip(labels,images,i_names,l_names,i_path,l_path)

True

# créer image et label histo_equalizer

In [27]:
def relabel_eq(labels,images,i_names,l_names,i_path,l_path):
    for i in range(len(images)):
        hist, bins = np.histogram(images[i].flatten(),256,[0,256])
        cdf = hist.cumsum()
        cdf_m = np.ma.masked_equal(cdf,0)
        cdf_m = (cdf_m - cdf_m.min())*255/(cdf_m.max()-cdf_m.min())
        cdf = np.ma.filled(cdf_m,0).astype('uint8')
        img_eq = cdf[images[i]]
        imgpil = Image.fromarray(img_eq) # Transformation du tableau en image PIL
        imgpil.save(i_path+l_names[i][:-4]+'_eq.bmp')

    new_labels = labels
    for i in range(len(labels)):
        with open(l_path+l_names[i][:-4]+'_eq.txt','w') as fp:
            ligne = f'{new_labels[i][0]} {new_labels[i][1]} {new_labels[i][2]} {new_labels[i][3]} {new_labels[i][4]}'
            fp.write(ligne)
            
    return True

In [29]:
i_path = '../piford/'
l_path = '../piford/'
relabel_eq(labels,images,i_names,l_names,i_path,l_path)

True

# Creer image + lable histo stretching

In [30]:
def relabel_stretch(labels,images,i_names,l_names,i_path,l_path):
    for i in range(len(images)):
        # Open the image and convert it to grayscale
        img_data = images[i]
        # Perform histogram stretching
        min_val = np.min(img_data)
        max_val = np.max(img_data)
        stretched_data = 255 * ((img_data - min_val) / (max_val - min_val))
        stretched_data = stretched_data.astype(np.uint8)
        # Save the stretched image to a new file
        imgpil = Image.fromarray(stretched_data) # Transformation du tableau en image PIL
        imgpil.save(i_path+l_names[i][:-4]+'_stretch.bmp')


    new_labels = labels
    for i in range(len(labels)):
        with open(l_path+l_names[i][:-4]+'_stretch.txt','w') as fp:
            ligne = f'{new_labels[i][0]} {new_labels[i][1]} {new_labels[i][2]} {new_labels[i][3]} {new_labels[i][4]}'
            fp.write(ligne)
            
    return True

In [31]:
i_path = '../piford/'
l_path = '../piford/'
relabel_stretch(labels,images,i_names,l_names,i_path,l_path)

True

# creer image + label gaussian noise

In [33]:
def relabel_gauss(labels,images,i_names,l_names,i_path,l_path,s=3):
    for i in range(len(images)):
        aug = iaa.imgcorruptlike.GaussianNoise(severity=s)
        img = aug.augment_image(images[i])
        imgpil = Image.fromarray(img) # Transformation du tableau en image PIL
        imgpil.save(i_path+l_names[i][:-4]+'_gauss.bmp')

    new_labels = labels
    for i in range(len(labels)):
        with open(l_path+l_names[i][:-4]+'_gauss.txt','w') as fp:
            ligne = f'{new_labels[i][0]} {new_labels[i][1]} {new_labels[i][2]} {new_labels[i][3]} {new_labels[i][4]}'
            fp.write(ligne)
            
    return True


In [34]:
i_path = '../piford/'
l_path = '../piford/'
relabel_gauss(labels,images,i_names,l_names,i_path,l_path,s=3)

True

# créer image + label Motion

In [35]:
def relabel_motion(labels,images,i_names,l_names,i_path,l_path,k=30):
    for i in range(len(images)):
        aug = iaa.MotionBlur(k)
        img = aug.augment_image(images[i])
        imgpil = Image.fromarray(img) # Transformation du tableau en image PIL
        imgpil.save(i_path+l_names[i][:-4]+'_motion.bmp')

    new_labels = labels
    for i in range(len(labels)):
        with open(l_path+l_names[i][:-4]+'_motion.txt','w') as fp:
            ligne = f'{new_labels[i][0]} {new_labels[i][1]} {new_labels[i][2]} {new_labels[i][3]} {new_labels[i][4]}'
            fp.write(ligne)
            
    return True

In [36]:
i_path = '../piford/'
l_path = '../piford/'
relabel_motion(labels,images,i_names,l_names,i_path,l_path,k=30)

True

# Créer image + label Blur

In [37]:
def relabel_blur(labels,images,i_names,l_names,i_path,l_path,s=3):
    for i in range(len(images)):
        aug = iaa.imgcorruptlike.GaussianBlur(severity=2)
        img = aug.augment_image(images[i])
        imgpil = Image.fromarray(img) # Transformation du tableau en image PIL
        imgpil.save(i_path+l_names[i][:-4]+'_blur.bmp')

    new_labels = labels
    for i in range(len(labels)):
        with open(l_path+l_names[i][:-4]+'_blur.txt','w') as fp:
            ligne = f'{new_labels[i][0]} {new_labels[i][1]} {new_labels[i][2]} {new_labels[i][3]} {new_labels[i][4]}'
            fp.write(ligne)
            
    return True

In [ ]:
i_path = '../piford/'
l_path = '../piford/'
relabel_blur(labels,images,i_names,l_names,i_path,l_path,s=3)